In [ ]:
# Import all pacakges needed for notebook
from kaggle.api.kaggle_api_extended import KaggleApi
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#load data to a dataframe using an API call
api = KaggleApi()
api.authenticate()
api.dataset_download_file('thedevastator/cancer-patients-and-air-pollution-a-new-link','cancer patient data sets.csv', path='./')
df = pd.read_csv('cancer%20patient%20data%20sets.csv')
df

In [ ]:
#Correlation Matrix, saved as PNG
corr = df.corr()
sns.set(rc={'figure.figsize':(25,15)})
sns.heatmap(corr, annot=True)
plt.savefig('Graphs\CorrPlot.png')
plt.show()

In [ ]:
#Basic Statistics
pd.set_option('display.max_columns', None)
print('Top 5 Rows of Dataset:')
display(df.head())
print('Descriptive Statistics:')
display(df.describe())
print('Missing Values:')
display(df.isnull().sum())
pd.reset_option('max_columns')


In [ ]:
#Distribution of values by feature
sns.set(rc={'figure.figsize':(20,5)})
fig, ax =plt.subplots(1,3)
sns.histplot(df['Age'], kde=False, ax=ax[0])
sns.histplot(df['Gender'], kde=False, ax=ax[1])
sns.histplot(df['Air Pollution'], kde=False, ax=ax[2])
plt.savefig('Graphs\Hist_Age_Gender_Pollution.png')
plt.show()

sns.set(rc={'figure.figsize':(20,5)})
fig, ax =plt.subplots(1,3)
sns.histplot(df['Alcohol use'], kde=False, ax=ax[0])
sns.histplot(df['Dust Allergy'], kde=False, ax=ax[1])
sns.histplot(df['OccuPational Hazards'], kde=False, ax=ax[2])
plt.savefig('Graphs\Hist_Alc_Dust_Hazards.png')
plt.show()


sns.set(rc={'figure.figsize':(20,5)})
fig, ax =plt.subplots(1,3)
sns.histplot(df['Genetic Risk'], kde=False, ax=ax[0])
sns.histplot(df['chronic Lung Disease'], kde=False, ax=ax[1])
sns.histplot(df['Balanced Diet'], kde=False, ax=ax[2])
plt.savefig('Graphs\Hist_Genetic_Lung_Diet.png')
plt.show()

sns.set(rc={'figure.figsize':(20,5)})
fig, ax =plt.subplots(1,3)
sns.histplot(df['Obesity'], kde=False, ax=ax[0])
sns.histplot(df['Smoking'], kde=False, ax=ax[1])
sns.histplot(df['Passive Smoker'], kde=False, ax=ax[2])
plt.savefig('Graphs\Hist_Obesity_Smoking_Passive.png')
plt.show()

sns.set(rc={'figure.figsize':(20,5)})
fig, ax =plt.subplots(1,3)
sns.histplot(df['Chest Pain'], kde=False, ax=ax[0])
sns.histplot(df['Coughing of Blood'], kde=False, ax=ax[1])
sns.histplot(df['Fatigue'], kde=False, ax=ax[2])
plt.savefig('Graphs\Hist_Chest_Blood_Fatigue.png')
plt.show()

sns.set(rc={'figure.figsize':(20,5)})
fig, ax =plt.subplots(1,3)
sns.histplot(df['Weight Loss'], kde=False, ax=ax[0])
sns.histplot(df['Shortness of Breath'], kde=False, ax=ax[1])
sns.histplot(df['Wheezing'], kde=False, ax=ax[2])
plt.savefig('Graphs\Hist_Weight_Breath_Wheezing.png')
plt.show()

sns.set(rc={'figure.figsize':(20,5)})
fig, ax =plt.subplots(1,3)
sns.histplot(df['Swallowing Difficulty'], kde=False, ax=ax[0])
sns.histplot(df['Clubbing of Finger Nails'], kde=False, ax=ax[1])
sns.histplot(df['Frequent Cold'], kde=False, ax=ax[2])
plt.savefig('Graphs\Hist_Swallowing_Nails_Cold.png')
plt.show()

sns.set(rc={'figure.figsize':(20,5)})
fig, ax =plt.subplots(1,3)
sns.histplot(df['Dry Cough'], kde=False, ax=ax[0])
sns.histplot(df['Snoring'], kde=False, ax=ax[1])
sns.histplot(df['Level'], kde=False, ax=ax[2])
plt.savefig('Graphs\Hist_Cough_Snoring_Level.png')
plt.show()

In [ ]:
# Stacked Bar Charts Showing the distribution of Level
binned_df = df.copy()
binned_df['Age'] = pd.cut(binned_df['Age'], 8)
order = ['Low','Medium','High']
sns.set(rc={'figure.figsize':(20,5)})
fig, ax =plt.subplots(1,3)
pd.crosstab(index=binned_df['Age'],columns=binned_df['Level'],normalize="index").reindex(order, axis="columns").plot(kind='bar',stacked=True,color=['#367E18','#F57328','#CC3636'],ax=ax[0])
pd.crosstab(index=binned_df['Gender'],columns=binned_df['Level'],normalize="index").reindex(order, axis="columns").plot(kind='bar',stacked=True,color=['#367E18','#F57328','#CC3636'],ax=ax[1])
pd.crosstab(index=binned_df['Air Pollution'],columns=binned_df['Level'],normalize="index").reindex(order, axis="columns").plot(kind='bar',stacked=True,color=['#367E18','#F57328','#CC3636'],ax=ax[2])
plt.savefig('Graphs\StkBr_Age_Gender_Pollution.png') 
plt.show()

sns.set(rc={'figure.figsize':(20,5)})
fig, ax =plt.subplots(1,3)
pd.crosstab(index=binned_df['Alcohol use'],columns=binned_df['Level'],normalize="index").reindex(order, axis="columns").plot(kind='bar',stacked=True,color=['#367E18','#F57328','#CC3636'],ax=ax[0])
pd.crosstab(index=binned_df['Dust Allergy'],columns=binned_df['Level'],normalize="index").reindex(order, axis="columns").plot(kind='bar',stacked=True,color=['#367E18','#F57328','#CC3636'],ax=ax[1])
pd.crosstab(index=binned_df['OccuPational Hazards'],columns=binned_df['Level'],normalize="index").reindex(order, axis="columns").plot(kind='bar',stacked=True,color=['#367E18','#F57328','#CC3636'],ax=ax[2])
plt.savefig('Graphs\StkBr_Alc_Dust_Hazards.png')
plt.show()

sns.set(rc={'figure.figsize':(20,5)})
fig, ax =plt.subplots(1,3)
pd.crosstab(index=binned_df['Genetic Risk'],columns=binned_df['Level'],normalize="index").reindex(order, axis="columns").plot(kind='bar',stacked=True,color=['#367E18','#F57328','#CC3636'],ax=ax[0])
pd.crosstab(index=binned_df['chronic Lung Disease'],columns=binned_df['Level'],normalize="index").reindex(order, axis="columns").plot(kind='bar',stacked=True,color=['#367E18','#F57328','#CC3636'],ax=ax[1])
pd.crosstab(index=binned_df['Balanced Diet'],columns=binned_df['Level'],normalize="index").reindex(order, axis="columns").plot(kind='bar',stacked=True,color=['#367E18','#F57328','#CC3636'],ax=ax[2])
plt.savefig('Graphs\StkBr_Genetic_Lung_Diet.png')
plt.show()

sns.set(rc={'figure.figsize':(20,5)})
fig, ax =plt.subplots(1,3)
pd.crosstab(index=binned_df['Obesity'],columns=binned_df['Level'],normalize="index").reindex(order, axis="columns").plot(kind='bar',stacked=True,color=['#367E18','#F57328','#CC3636'],ax=ax[0])
pd.crosstab(index=binned_df['Smoking'],columns=binned_df['Level'],normalize="index").reindex(order, axis="columns").plot(kind='bar',stacked=True,color=['#367E18','#F57328','#CC3636'],ax=ax[1])
pd.crosstab(index=binned_df['Passive Smoker'],columns=binned_df['Level'],normalize="index").reindex(order, axis="columns").plot(kind='bar',stacked=True,color=['#367E18','#F57328','#CC3636'],ax=ax[2])
plt.savefig('Graphs\StkBr_Obesity_Smoking_Passive.png')
plt.show()

sns.set(rc={'figure.figsize':(20,5)})
fig, ax =plt.subplots(1,3)
pd.crosstab(index=binned_df['Chest Pain'],columns=binned_df['Level'],normalize="index").reindex(order, axis="columns").plot(kind='bar',stacked=True,color=['#367E18','#F57328','#CC3636'],ax=ax[0])
pd.crosstab(index=binned_df['Coughing of Blood'],columns=binned_df['Level'],normalize="index").reindex(order, axis="columns").plot(kind='bar',stacked=True,color=['#367E18','#F57328','#CC3636'],ax=ax[1])
pd.crosstab(index=binned_df['Fatigue'],columns=binned_df['Level'],normalize="index").reindex(order, axis="columns").plot(kind='bar',stacked=True,color=['#367E18','#F57328','#CC3636'],ax=ax[2])
plt.savefig('Graphs\StkBr_Chest_Blood_Fatigue.png')
plt.show()

sns.set(rc={'figure.figsize':(20,5)})
fig, ax =plt.subplots(1,3)
pd.crosstab(index=binned_df['Weight Loss'],columns=binned_df['Level'],normalize="index").reindex(order, axis="columns").plot(kind='bar',stacked=True,color=['#367E18','#F57328','#CC3636'],ax=ax[0])
pd.crosstab(index=binned_df['Shortness of Breath'],columns=binned_df['Level'],normalize="index").reindex(order, axis="columns").plot(kind='bar',stacked=True,color=['#367E18','#F57328','#CC3636'],ax=ax[1])
pd.crosstab(index=binned_df['Wheezing'],columns=binned_df['Level'],normalize="index").reindex(order, axis="columns").plot(kind='bar',stacked=True,color=['#367E18','#F57328','#CC3636'],ax=ax[2])
plt.savefig('Graphs\StkBr_Weight_Breath_Wheezing.png')
plt.show()

sns.set(rc={'figure.figsize':(20,5)})
fig, ax =plt.subplots(1,3)
pd.crosstab(index=binned_df['Swallowing Difficulty'],columns=binned_df['Level'],normalize="index").reindex(order, axis="columns").plot(kind='bar',stacked=True,color=['#367E18','#F57328','#CC3636'],ax=ax[0])
pd.crosstab(index=binned_df['Clubbing of Finger Nails'],columns=binned_df['Level'],normalize="index").reindex(order, axis="columns").plot(kind='bar',stacked=True,color=['#367E18','#F57328','#CC3636'],ax=ax[1])
pd.crosstab(index=binned_df['Frequent Cold'],columns=binned_df['Level'],normalize="index").reindex(order, axis="columns").plot(kind='bar',stacked=True,color=['#367E18','#F57328','#CC3636'],ax=ax[2])
plt.savefig('Graphs\StkBr_Swallowing_Nails_Cold.png')
plt.show()

sns.set(rc={'figure.figsize':(20,5)})
fig, ax =plt.subplots(1,2)
pd.crosstab(index=binned_df['Dry Cough'],columns=binned_df['Level'],normalize="index").reindex(order, axis="columns").plot(kind='bar',stacked=True,color=['#367E18','#F57328','#CC3636'],ax=ax[0])
pd.crosstab(index=binned_df['Snoring'],columns=binned_df['Level'],normalize="index").reindex(order, axis="columns").plot(kind='bar',stacked=True,color=['#367E18','#F57328','#CC3636'],ax=ax[1])
plt.savefig('Graphs\StkBr_Cough_Snoring.png')  
plt.show()

In [ ]:
# Stacked Bar Charts Showing the distribution of Level
two_level_df = df.copy()
two_level_df['Age'] = pd.cut(two_level_df['Age'], [0,18,35,55,100])
two_level_df['Level'] = two_level_df['Level'].map({'Low': 'Not High', 'Medium': 'Not High', 'High': 'High'})
order = ['Not High','High']
sns.set(rc={'figure.figsize':(20,5)})
fig, ax =plt.subplots(1,3)
pd.crosstab(index=two_level_df['Age'],columns=two_level_df['Level'],normalize="index").reindex(order, axis="columns").plot(kind='bar',stacked=True,color=['#367E18','#CC3636'],ax=ax[0])
pd.crosstab(index=two_level_df['Gender'],columns=two_level_df['Level'],normalize="index").reindex(order, axis="columns").plot(kind='bar',stacked=True,color=['#367E18','#CC3636'],ax=ax[1])
pd.crosstab(index=two_level_df['Air Pollution'],columns=two_level_df['Level'],normalize="index").reindex(order, axis="columns").plot(kind='bar',stacked=True,color=['#367E18','#CC3636'],ax=ax[2])
plt.savefig('Graphs\Grouped_StkBr_Age_Gender_Pollution.png') 
plt.show()

sns.set(rc={'figure.figsize':(20,5)})
fig, ax =plt.subplots(1,3)
pd.crosstab(index=two_level_df['Alcohol use'],columns=two_level_df['Level'],normalize="index").reindex(order, axis="columns").plot(kind='bar',stacked=True,color=['#367E18','#CC3636'],ax=ax[0])
pd.crosstab(index=two_level_df['Dust Allergy'],columns=two_level_df['Level'],normalize="index").reindex(order, axis="columns").plot(kind='bar',stacked=True,color=['#367E18','#CC3636'],ax=ax[1])
pd.crosstab(index=two_level_df['OccuPational Hazards'],columns=two_level_df['Level'],normalize="index").reindex(order, axis="columns").plot(kind='bar',stacked=True,color=['#367E18','#CC3636'],ax=ax[2])
plt.savefig('Graphs\Grouped_StkBr_Alc_Dust_Hazards.png')
plt.show()

sns.set(rc={'figure.figsize':(20,5)})
fig, ax =plt.subplots(1,3)
pd.crosstab(index=two_level_df['Genetic Risk'],columns=two_level_df['Level'],normalize="index").reindex(order, axis="columns").plot(kind='bar',stacked=True,color=['#367E18','#CC3636'],ax=ax[0])
pd.crosstab(index=two_level_df['chronic Lung Disease'],columns=two_level_df['Level'],normalize="index").reindex(order, axis="columns").plot(kind='bar',stacked=True,color=['#367E18','#CC3636'],ax=ax[1])
pd.crosstab(index=two_level_df['Balanced Diet'],columns=two_level_df['Level'],normalize="index").reindex(order, axis="columns").plot(kind='bar',stacked=True,color=['#367E18','#CC3636'],ax=ax[2])
plt.savefig('Graphs\Grouped_StkBr_Genetic_Lung_Diet.png')
plt.show()

sns.set(rc={'figure.figsize':(20,5)})
fig, ax =plt.subplots(1,3)
pd.crosstab(index=two_level_df['Obesity'],columns=two_level_df['Level'],normalize="index").reindex(order, axis="columns").plot(kind='bar',stacked=True,color=['#367E18','#CC3636'],ax=ax[0])
pd.crosstab(index=two_level_df['Smoking'],columns=two_level_df['Level'],normalize="index").reindex(order, axis="columns").plot(kind='bar',stacked=True,color=['#367E18','#CC3636'],ax=ax[1])
pd.crosstab(index=two_level_df['Passive Smoker'],columns=two_level_df['Level'],normalize="index").reindex(order, axis="columns").plot(kind='bar',stacked=True,color=['#367E18','#CC3636'],ax=ax[2])
plt.savefig('Graphs\Grouped_StkBr_Obesity_Smoking_Passive.png')
plt.show()

sns.set(rc={'figure.figsize':(20,5)})
fig, ax =plt.subplots(1,3)
pd.crosstab(index=two_level_df['Chest Pain'],columns=two_level_df['Level'],normalize="index").reindex(order, axis="columns").plot(kind='bar',stacked=True,color=['#367E18','#CC3636'],ax=ax[0])
pd.crosstab(index=two_level_df['Coughing of Blood'],columns=two_level_df['Level'],normalize="index").reindex(order, axis="columns").plot(kind='bar',stacked=True,color=['#367E18','#CC3636'],ax=ax[1])
pd.crosstab(index=two_level_df['Fatigue'],columns=two_level_df['Level'],normalize="index").reindex(order, axis="columns").plot(kind='bar',stacked=True,color=['#367E18','#CC3636'],ax=ax[2])
plt.savefig('Graphs\Grouped_StkBr_Chest_Blood_Fatigue.png')
plt.show()

sns.set(rc={'figure.figsize':(20,5)})
fig, ax =plt.subplots(1,3)
pd.crosstab(index=two_level_df['Weight Loss'],columns=two_level_df['Level'],normalize="index").reindex(order, axis="columns").plot(kind='bar',stacked=True,color=['#367E18','#CC3636'],ax=ax[0])
pd.crosstab(index=two_level_df['Shortness of Breath'],columns=two_level_df['Level'],normalize="index").reindex(order, axis="columns").plot(kind='bar',stacked=True,color=['#367E18','#CC3636'],ax=ax[1])
pd.crosstab(index=two_level_df['Wheezing'],columns=two_level_df['Level'],normalize="index").reindex(order, axis="columns").plot(kind='bar',stacked=True,color=['#367E18','#CC3636'],ax=ax[2])
plt.savefig('Graphs\Grouped_StkBr_Weight_Breath_Wheezing.png')
plt.show()

sns.set(rc={'figure.figsize':(20,5)})
fig, ax =plt.subplots(1,3)
pd.crosstab(index=two_level_df['Swallowing Difficulty'],columns=two_level_df['Level'],normalize="index").reindex(order, axis="columns").plot(kind='bar',stacked=True,color=['#367E18','#CC3636'],ax=ax[0])
pd.crosstab(index=two_level_df['Clubbing of Finger Nails'],columns=two_level_df['Level'],normalize="index").reindex(order, axis="columns").plot(kind='bar',stacked=True,color=['#367E18','#CC3636'],ax=ax[1])
pd.crosstab(index=two_level_df['Frequent Cold'],columns=two_level_df['Level'],normalize="index").reindex(order, axis="columns").plot(kind='bar',stacked=True,color=['#367E18','#CC3636'],ax=ax[2])
plt.savefig('Graphs\Grouped_StkBr_Swallowing_Nails_Cold.png')
plt.show()

sns.set(rc={'figure.figsize':(20,5)})
fig, ax =plt.subplots(1,2)
pd.crosstab(index=two_level_df['Dry Cough'],columns=two_level_df['Level'],normalize="index").reindex(order, axis="columns").plot(kind='bar',stacked=True,color=['#367E18','#CC3636'],ax=ax[0])
pd.crosstab(index=two_level_df['Snoring'],columns=two_level_df['Level'],normalize="index").reindex(order, axis="columns").plot(kind='bar',stacked=True,color=['#367E18','#CC3636'],ax=ax[1])
plt.savefig('Graphs\Grouped_StkBr_Cough_Snoring.png')  
plt.show()

In [ ]:
#  Create a score for separation, outliers, symmetry, distribution shape, and distribution spread.

# Separation:
def separation_score(df, predictor, response):
    """
    Calculates a separation score for a predictor variable with respect to the response variable
    """
    pivot_table = df.pivot_table(index=predictor, columns=response, aggfunc='size')
    pivot_table = pivot_table.div(pivot_table.sum(axis=1), axis=0)
    pivot_table = pivot_table.fillna(0)
    score = pivot_table.diff(axis=1).abs().sum().sum()
    return score

separation_scores = {}
for predictor in ['Age','Gender','Air Pollution','Alcohol use','Dust Allergy','OccuPational Hazards','Genetic Risk','chronic Lung Disease','Balanced Diet',
                    'Obesity' ,'Smoking' ,'Passive Smoker' ,'Chest Pain' ,'Coughing of Blood','Fatigue','Weight Loss' ,'Shortness of Breath',
                    'Wheezing','Swallowing Difficulty','Clubbing of Finger Nails','Frequent Cold','Dry Cough','Snoring']:
    separation_scores[predictor] = separation_score(df, predictor, 'Level')

print("Separation Scores:")
print(separation_scores)

#  The separation score measures the extent to which the levels of the response 
#  variable are separated for each predictor variable. A higher score indicates 
#  a stronger separation between the levels of the response variable.

In [ ]:
#Outliers
def detect_outliers(df):
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    IQR = Q3 - Q1
    outliers = ((df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR)))
    return outliers

outliers_detected = detect_outliers(df)

outliers_df = pd.DataFrame(outliers_detected.sum(), columns=['Num Outliers'])
outliers_df.sort_values(by='Num Outliers', ascending=False, inplace=True)

print("Outliers detected:")
print(outliers_df)

In [ ]:
#  Symmetry:

def symmetry_score(df, predictor):
    """
    Calculates a symmetry score for a predictor variable
    """
    mean = df[predictor].mean()
    median = df[predictor].median()
    score = abs(mean - median) / (mean + median)
    return score

symmetry_scores = {}
for predictor in ['Age','Gender','Air Pollution','Alcohol use','Dust Allergy','OccuPational Hazards','Genetic Risk','chronic Lung Disease','Balanced Diet',
                    'Obesity' ,'Smoking' ,'Passive Smoker' ,'Chest Pain' ,'Coughing of Blood','Fatigue','Weight Loss' ,'Shortness of Breath',
                    'Wheezing','Swallowing Difficulty','Clubbing of Finger Nails','Frequent Cold','Dry Cough','Snoring']:
    symmetry_scores[predictor] = symmetry_score(df, predictor)

print("Symmetry Scores:")
print(symmetry_scores)

#  The symmetry score measures the extent to which the mean and median are 
#  different for each predictor variable.  A lower score indicates a more 
#  symmetrical distribution, while a higher score indicates a more skewed 
#  distribution.

In [ ]:
#  Kurtosis:

def kurtosis_score(df, predictor):
    """
    Calculates a kurtosis score for a predictor variable
    """
    kurtosis = df[predictor].kurtosis()
    score = kurtosis / df[predictor].std()**4
    return score

kurtosis_scores = {}
for predictor in ['Age','Gender','Air Pollution','Alcohol use','Dust Allergy','OccuPational Hazards','Genetic Risk','chronic Lung Disease','Balanced Diet',
                    'Obesity' ,'Smoking' ,'Passive Smoker' ,'Chest Pain' ,'Coughing of Blood','Fatigue','Weight Loss' ,'Shortness of Breath',
                    'Wheezing','Swallowing Difficulty','Clubbing of Finger Nails','Frequent Cold','Dry Cough','Snoring']:
    kurtosis_scores[predictor] = kurtosis_score(df, predictor)

print("Kurtosis Scores:")
print(kurtosis_scores)

#  The kurtosis score measures the extent to which the distribution of each predictor 
#  variable is peaked or flat relative to a normal distribution. A lower score 
#  indicates a more normal distribution, while a higher score indicates a more peaked or 
#  flat distribution.

In [ ]:
# Calculate the correlation matrix
corr_df = df.copy()
corr_df['Level'] = corr_df['Level'].map({'Low': 1, 'Medium': 2, 'High': 3})
corr = corr_df.corr()

# Create a table of the correlation strengths
corr_table = corr[['Level']].loc[['Age','Gender','Air Pollution','Alcohol use','Dust Allergy','OccuPational Hazards','Genetic Risk','chronic Lung Disease','Balanced Diet',
                                'Obesity' ,'Smoking' ,'Passive Smoker' ,'Chest Pain' ,'Coughing of Blood','Fatigue','Weight Loss' ,'Shortness of Breath',
                                'Wheezing','Swallowing Difficulty','Clubbing of Finger Nails','Frequent Cold','Dry Cough','Snoring']]
corr_table = corr_table.rename(columns={'Level': 'Correlation Strength'})

print("Correlation Table:")
print(corr_table)

In [ ]:
##Train Test Split
from sklearn.model_selection import train_test_split
df1 = df.drop(['index', 'Patient Id'], axis=1)
X = df.drop(['index','Level', 'Patient Id'], axis=1)
y = df['Level']

display(X.head())
display(y.head())

# using the train test split function
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=104, test_size=0.25, shuffle=True)

In [ ]:
# Random Forest and Boruta
from sklearn.ensemble import RandomForestClassifier

# define random forest classifier
forest = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=5)
forest.fit(X_train, y_train)
from boruta import BorutaPy
# define Boruta feature selection method
feat_selector = BorutaPy(forest, n_estimators='auto', verbose=2, random_state=113)
# find all relevant features
feat_selector.fit(np.array(X_train), np.array(y_train))
# check selected features
display(feat_selector.support_)
# check ranking of features
display(feat_selector.ranking_)
# call transform() on X to filter it down to selected features
X_filtered = feat_selector.transform(np.array(X_train))

In [ ]:
# zip my names, ranks, and decisions in a single iterable
feature_ranks = list(zip(df1.columns, 
                         feat_selector.ranking_, 
                         feat_selector.support_))

# iterate through and print out the results
for feat in feature_ranks:
    print('Feature: {:<25} Rank: {},  Keep: {}'.format(feat[0], feat[1], feat[2]))
feat_selector

In [37]:
#Logistic Regression Tranformations
log_df = df.copy()
log_df['Level'] = log_df['Level'].map({'Low': 0, 'Medium': 0, 'High': 1}) #Two levels (High and Not High)

##Train Test Split
from sklearn.model_selection import train_test_split
X = log_df.drop(['index','Level', 'Patient Id'], axis=1)
y = log_df['Level']

display(X.head())
display(y.head())

# using the train test split function
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=104, test_size=0.25, shuffle=True)

,Age,Gender,Air Pollution,Alcohol use,Dust Allergy,OccuPational Hazards,Genetic Risk,chronic Lung Disease,Balanced Diet,Obesity,...,Coughing of Blood,Fatigue,Weight Loss,Shortness of Breath,Wheezing,Swallowing Difficulty,Clubbing of Finger Nails,Frequent Cold,Dry Cough,Snoring
0,33,1,2,4,5,4,3,2,2,4,...,4,3,4,2,2,3,1,2,3,4
1,17,1,3,1,5,3,4,2,2,2,...,3,1,3,7,8,6,2,1,7,2
2,35,1,4,5,6,5,5,4,6,7,...,8,8,7,9,2,1,4,6,7,2
3,37,1,7,7,7,7,6,7,7,7,...,8,4,2,3,1,4,5,6,7,5
4,46,1,6,8,7,7,7,6,7,7,...,9,3,2,4,1,4,2,4,2,3


0    0
1    0
2    1
3    1
4    1
Name: Level, dtype: int64

In [38]:
#Baseline Model creation
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

# Create a logistic regression model
logreg = LogisticRegression(max_iter=4000)
# fitting our data onto our model
logreg.fit(X_train, y_train)
# using our model to predict our y values with our test set data
y_pred_class = logreg.predict(X_test)
y_pred_train = logreg.predict(X_train)
# predict probabilities for our testing target variable using our model
logreg.predict_proba(X_test)

# checking accuracy
print('Test Accuracy score: ', metrics.accuracy_score(y_test, y_pred_class))
print('Train Accuracy score: ', metrics.accuracy_score(y_train, y_pred_train))
# checking precision
print('Test Precision score: ', metrics.precision_score(y_test, y_pred_class))
print('Train Precision score: ', metrics.precision_score(y_train, y_pred_train))
# checking Recall (Sensitivity)
print('Test Recall (Sensitivity) score: ', metrics.recall_score(y_test, y_pred_class))
print('Train Recall (Sensitivity) score: ', metrics.recall_score(y_train, y_pred_train))
# checking F1 score
print('Test F1 score: ', metrics.f1_score(y_test, y_pred_class))
print('Train F1 score: ', metrics.f1_score(y_train, y_pred_train))

Test Accuracy score:  1.0
Train Accuracy score:  1.0
Test Precision score:  1.0
Train Precision score:  1.0
Test Recall (Sensitivity) score:  1.0
Train Recall (Sensitivity) score:  1.0
Test F1 score:  1.0
Train F1 score:  1.0


In [39]:
from sklearn.feature_selection import RFE

rfe=RFE(logreg, n_features_to_select=2, verbose=5)
rfe.fit(X_train, y_train)
y_pred_train=rfe.predict(X_train)
y_pred_class=rfe.predict(X_test)

print('Test Accuracy score: ', metrics.accuracy_score(y_test, y_pred_class))
print('Train Accuracy score: ', metrics.accuracy_score(y_train, y_pred_train))
# checking precision
print('Test Precision score: ', metrics.precision_score(y_test, y_pred_class))
print('Train Precision score: ', metrics.precision_score(y_train, y_pred_train))
# checking Recall (Sensitivity)
print('Test Recall (Sensitivity) score: ', metrics.recall_score(y_test, y_pred_class))
print('Train Recall (Sensitivity) score: ', metrics.recall_score(y_train, y_pred_train))
# checking F1 score
print('Test F1 score: ', metrics.f1_score(y_test, y_pred_class))
print('Train F1 score: ', metrics.f1_score(y_train, y_pred_train))

Fitting estimator with 23 features.
Fitting estimator with 22 features.
Fitting estimator with 21 features.
Fitting estimator with 20 features.
Fitting estimator with 19 features.
Fitting estimator with 18 features.
Fitting estimator with 17 features.
Fitting estimator with 16 features.
Fitting estimator with 15 features.
Fitting estimator with 14 features.
Fitting estimator with 13 features.
Fitting estimator with 12 features.
Fitting estimator with 11 features.
Fitting estimator with 10 features.
Fitting estimator with 9 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 3 features.
Test Accuracy score:  0.984
Train Accuracy score:  0.9653333333333334
Test Precision score:  1.0
Train Precision score:  1.0
Test Recall (Sensitivity) score:  0.9574468085106383
Train Recall (Sensitivity) score:  0.9040590405904059
Test F1 score:  0.9

In [40]:
#Scale Data and retest
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create a logistic regression model
logreg = LogisticRegression(max_iter=400)
# fitting our data onto our model
logreg.fit(X_train_scaled, y_train)
# using our model to predict our y values with our test set data
y_pred_class = logreg.predict(X_test_scaled)
y_pred_train = logreg.predict(X_train_scaled)
# predict probabilities for our testing target variable using our model
logreg.predict_proba(X_test_scaled)

# checking accuracy
print('Test Accuracy score: ', metrics.accuracy_score(y_test, y_pred_class))
print('Train Accuracy score: ', metrics.accuracy_score(y_train, y_pred_train))
# checking precision
print('Test Precision score: ', metrics.precision_score(y_test, y_pred_class))
print('Train Precision score: ', metrics.precision_score(y_train, y_pred_train))
# checking Recall (Sensitivity)
print('Test Recall (Sensitivity) score: ', metrics.recall_score(y_test, y_pred_class))
print('Train Recall (Sensitivity) score: ', metrics.recall_score(y_train, y_pred_train))
# checking F1 score
print('Test F1 score: ', metrics.f1_score(y_test, y_pred_class))
print('Train F1 score: ', metrics.f1_score(y_train, y_pred_train))

Test Accuracy score:  1.0
Train Accuracy score:  1.0
Test Precision score:  1.0
Train Precision score:  1.0
Test Recall (Sensitivity) score:  1.0
Train Recall (Sensitivity) score:  1.0
Test F1 score:  1.0
Train F1 score:  1.0


In [45]:
from sklearn.feature_selection import RFE

rfe=RFE(logreg, n_features_to_select=4, verbose=5)
rfe.fit(X_train_scaled, y_train)
y_pred_train=rfe.predict(X_train_scaled)
y_pred_class=rfe.predict(X_test_scaled)

print('Test Accuracy score: ', metrics.accuracy_score(y_test, y_pred_class))
print('Train Accuracy score: ', metrics.accuracy_score(y_train, y_pred_train))
# checking precision
print('Test Precision score: ', metrics.precision_score(y_test, y_pred_class))
print('Train Precision score: ', metrics.precision_score(y_train, y_pred_train))
# checking Recall (Sensitivity)
print('Test Recall (Sensitivity) score: ', metrics.recall_score(y_test, y_pred_class))
print('Train Recall (Sensitivity) score: ', metrics.recall_score(y_train, y_pred_train))
# checking F1 score
print('Test F1 score: ', metrics.f1_score(y_test, y_pred_class))
print('Train F1 score: ', metrics.f1_score(y_train, y_pred_train))

Fitting estimator with 23 features.
Fitting estimator with 22 features.
Fitting estimator with 21 features.
Fitting estimator with 20 features.
Fitting estimator with 19 features.
Fitting estimator with 18 features.
Fitting estimator with 17 features.
Fitting estimator with 16 features.
Fitting estimator with 15 features.
Fitting estimator with 14 features.
Fitting estimator with 13 features.
Fitting estimator with 12 features.
Fitting estimator with 11 features.
Fitting estimator with 10 features.
Fitting estimator with 9 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Test Accuracy score:  0.996
Train Accuracy score:  0.988
Test Precision score:  0.9894736842105263
Train Precision score:  0.9678571428571429
Test Recall (Sensitivity) score:  1.0
Train Recall (Sensitivity) score:  1.0
Test F1 score:  0.9947089947089947
Train F1 score:  0.983666061705989


In [59]:
# Applying PCA function on training
# and testing set of X component
from sklearn.decomposition import PCA
 
pca = PCA(.95)
 
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)
 
logregPCA = LogisticRegression()
logregPCA.fit(X_train_pca,y_train)
y_pred_train=logregPCA.predict(X_train_pca)
y_pred_class=logregPCA.predict(X_test_pca)

print('Test Accuracy score: ', metrics.accuracy_score(y_test, y_pred_class))
print('Train Accuracy score: ', metrics.accuracy_score(y_train, y_pred_train))
# checking precision
print('Test Precision score: ', metrics.precision_score(y_test, y_pred_class))
print('Train Precision score: ', metrics.precision_score(y_train, y_pred_train))
# checking Recall (Sensitivity)
print('Test Recall (Sensitivity) score: ', metrics.recall_score(y_test, y_pred_class))
print('Train Recall (Sensitivity) score: ', metrics.recall_score(y_train, y_pred_train))
# checking F1 score
print('Test F1 score: ', metrics.f1_score(y_test, y_pred_class))
print('Train F1 score: ', metrics.f1_score(y_train, y_pred_train))

Test Accuracy score:  0.996
Train Accuracy score:  0.988
Test Precision score:  0.9894736842105263
Train Precision score:  0.9678571428571429
Test Recall (Sensitivity) score:  1.0
Train Recall (Sensitivity) score:  1.0
Test F1 score:  0.9947089947089947
Train F1 score:  0.983666061705989


In [60]:
# Applying PCA function on training
# and testing set of X component
from sklearn.decomposition import PCA
 
pca = PCA(n_components=2)
 
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)
 
logregPCA = LogisticRegression()
logregPCA.fit(X_train_pca,y_train)
y_pred_train=logregPCA.predict(X_train_pca)
y_pred_class=logregPCA.predict(X_test_pca)

print('Test Accuracy score: ', metrics.accuracy_score(y_test, y_pred_class))
print('Train Accuracy score: ', metrics.accuracy_score(y_train, y_pred_train))
# checking precision
print('Test Precision score: ', metrics.precision_score(y_test, y_pred_class))
print('Train Precision score: ', metrics.precision_score(y_train, y_pred_train))
# checking Recall (Sensitivity)
print('Test Recall (Sensitivity) score: ', metrics.recall_score(y_test, y_pred_class))
print('Train Recall (Sensitivity) score: ', metrics.recall_score(y_train, y_pred_train))
# checking F1 score
print('Test F1 score: ', metrics.f1_score(y_test, y_pred_class))
print('Train F1 score: ', metrics.f1_score(y_train, y_pred_train))

Test Accuracy score:  0.968
Train Accuracy score:  0.9826666666666667
Test Precision score:  0.9886363636363636
Train Precision score:  1.0
Test Recall (Sensitivity) score:  0.925531914893617
Train Recall (Sensitivity) score:  0.9520295202952029
Test F1 score:  0.9560439560439561
Train F1 score:  0.9754253308128544
